## 1. Project Background
This project aims to use textblob to perform textual analysis on reviews of Airbnb listings in Tokyo in March 2019. The data is download from InsideAirbnb, an independent, non-commercial set of tools and data that allows you to explore how Airbnb is really being used in cities around the world. 

Each listing will have more than one reviews. We will obtain the polarity, subjectivity and length of each review in terms of number of words for each review, and aggregate them on a listing level. Purpose of ob

## 2. Understanding and Preparing Data

In [1]:
import pandas as pd
from textblob import TextBlob

In [3]:
df_reviews_march = pd.read_csv("tokyoreviews_March2019_with_lang_final.csv")

/Users/CelineWong/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (0,1) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


The review data contains more than 300,000 observations for March data alone. All data columns, excep language, are extracted from InsideAirbnb. We precess the language of each review using another program, which will noe be in-scope in this case. Note that TextBlob does not work on langauges other than English in this case.

In [4]:
#understand the data type of each column
df_reviews_march.dtypes

listing_id        object
id                object
date              object
reviewer_id      float64
reviewer_name     object
comments          object
language          object
dtype: object

In [6]:
#count the number for each unique value in language 
df_reviews_jun["language"].value_counts()

en       190695
ko        33571
zh-cn     32114
ja        23595
fr         5962
es         2556
zh-tw      1148
de         1136
so         1125
th          929
ro          807
it          646
            584
af          563
ru          316
pt          299
id          294
pl          293
nl          266
cs          259
ca          204
no          145
vi          121
sv          116
tl          103
da           85
cy           85
fi           78
hr           76
sw           54
et           53
sk           39
hu           34
tr           23
sq           20
sl           19
ar            8
lt            7
lv            5
el            4
bg            3
mk            2
he            2
uk            1
Name: language, dtype: int64

To obtain the length, subjectivity and polarity for each review, we iterate each observation by their review id and combine thern into a list, which is further transformed to a panda dataframe

In [7]:
#Create a list which contain the unique review id, polarity and subjectivity score for each review for March Data.
mar=[]
for index1, row1 in df_reviews_march.iterrows():
    review_id=row1['id']
    reviews = row1['comments']
    reviews = str(reviews)
    lengrev = len(reviews.split())    
    testimonial = TextBlob(reviews)
    polarity=testimonial.sentiment.polarity
    subjectivity=testimonial.sentiment.subjectivity
    mar.append({'Review_id':review_id,'Review_Length':lengrev, 'Polarity':polarity, 'Subjectivity': subjectivity})

In [11]:
#Transform the list to pandas dataframe
mardf=pd.DataFrame(mar)

In [12]:
#Rearrange the column sequence
mardf = mardf[['Review_id','Review_Length','Polarity', 'Subjectivity']]

We merge the data frame we just created with the main table by review id. After merging the two dataframes, we aggregate the review score on listing level and save the result for further analysis.

In [14]:
#combine the dataframe which contains the subjectivity and polarity score to the main table table, using the review ID as the key
merge_reviews_mar = pd.merge(df_reviews_march,mardf,left_on=['id'], right_on=['Review_id'],how='outer')


In [29]:
merge_reviews_mar['listing_id'] = merge_reviews_mar.listing_id.astype(int)

In [31]:
#Save the combined result
export_csv = merge_reviews_mar.to_csv(r'march_review_wanalysis.csv',index=None, header=True)


In [33]:
#Calculate the mean polarity and subjectivity of every listing, group by listing ID
agg_march=merge_reviews_mar.groupby('listing_id').agg({'Review_Length':'mean','Polarity':'mean','Subjectivity':'mean' }).reset_index()


In [34]:
export_csv=agg_march.to_csv(r'march_review_score.csv',index=None, header=True)
